In [1]:
import tensorflow as tf

from tensorflow.keras import optimizers, datasets, layers, models

(train_image, train_label), (test_image, test_label) = datasets.fashion_mnist.load_data()

In [2]:
import numpy as np

ReScale = lambda img : tf.cast(img, tf.float32) / 255.0

train_image = ReScale(train_image)

np.min(train_image[0]), np.max(train_image[0])

(0.0, 1.0)

In [3]:
test_image = ReScale(test_image)

np.min(test_image[0]), np.max(test_image[0])

(0.0, 1.0)

In [4]:
import warnings; warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential

input_shape = (28, 28)

model = Sequential()

model.add(layers.Flatten(input_shape=input_shape))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [5]:
loss = 'sparse_categorical_crossentropy'

model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

hist = model.fit(train_image, train_label, epochs=8, validation_data=(test_image, test_label), verbose=False)

In [6]:
model.evaluate(train_image, train_label, verbose=False)

[0.23804238438606262, 0.9101333618164062]

In [7]:
model.evaluate(test_image, test_label, verbose=False)

[0.34085509181022644, 0.8813999891281128]

In [8]:
import keras_tuner as kt

def model_builder(hp):
  """
  Keras Tuner Model Builder
  """
  model = Sequential()

  model.add(layers.Flatten(input_shape=input_shape))

  units = hp.Int('units', min_value=16, max_value=48, step=16)

  model.add(layers.Dense(units, activation='relu'))

  model.add(layers.Dense(10, activation='softmax'))

  choices = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

  optim = optimizers.Adam(learning_rate=choices)

  model.compile(optimizer=optim, loss=loss, metrics=['accuracy'])

  return model

In [ ]:
target = 'val_accuracy'

project = 'Belajar Keras Tuner'

tuner = kt.Hyperband(model_builder, objective=target, max_epochs=8, factor=3, project_name=project)

tuner.search(train_image, train_label, epochs=8, validation_data=(test_image, test_label))

In [10]:
resulta = tuner.get_best_hyperparameters()[0]

resulta.get('units')

48

In [11]:
resulta.get('learning_rate')

0.001

In [12]:
best_models = tuner.get_best_models(num_models=1)[0]

best_models.evaluate(train_image, train_label, verbose=False)

[0.2908763587474823, 0.8932666778564453]

In [13]:
best_models.evaluate(test_image, test_label, verbose=False)

[0.3684285879135132, 0.8684999942779541]